In [8]:
import torch
from brt._C.router import generate_indices_and_loads, dispatch_with_indices_and_loads,dispatch_with_dst_indices_1d

path_num = 4
gates = torch.randn((8, path_num)).cuda()
topk_indices = torch.topk(gates, k=1, dim=1).indices

hot_mask = (
    torch.zeros(gates.size(0), path_num, dtype=torch.int32, device=gates.device,)
    .scatter_(1, topk_indices, 1)
    .cuda()
)

supported_capacities = torch.Tensor([1, 2, 4, 8, 16]).int().cuda()
# supported_capacities = None

route_indices, dst_loads = generate_indices_and_loads(
    hot_mask, supported_capacities, capacity_padding=True, is_dst_index=True
)
print(route_indices)
print(dst_loads)


tensor([[0, 0, 1, 0],
        [0, 0, 2, 0],
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 0, 1],
        [2, 0, 0, 0],
        [0, 0, 3, 0],
        [0, 0, 4, 0]], device='cuda:0', dtype=torch.int32)
tensor([2, 1, 4, 1], device='cuda:0', dtype=torch.int32)


In [12]:
in_data = torch.randn((8, path_num)).cuda()
print(in_data)
out_data_1 = dispatch_with_indices_and_loads(in_data, route_indices, dst_loads)
print(out_data_1)
out_data_2 = dispatch_with_dst_indices_1d(in_data, route_indices, dst_loads)
print(out_data_2)
torch.allclose(out_data_1[0], out_data_2)

tensor([[ 1.5505,  0.4794,  0.2362, -0.3262],
        [ 1.4210,  0.9824,  1.0255, -1.0544],
        [-2.0510, -1.7376,  0.6509,  0.1170],
        [-1.1487,  0.4429, -0.4612,  0.5667],
        [ 0.2725, -0.2631, -0.7710, -0.0592],
        [-0.8056,  0.3006, -1.3536,  1.1864],
        [ 0.4708, -0.0932,  0.2063, -1.8165],
        [-0.0180,  0.6837, -1.8995,  0.4710]], device='cuda:0')
[tensor([[-2.0510, -1.7376,  0.6509,  0.1170],
        [-0.8056,  0.3006, -1.3536,  1.1864],
        [-1.1487,  0.4429, -0.4612,  0.5667],
        [ 1.5505,  0.4794,  0.2362, -0.3262],
        [ 1.4210,  0.9824,  1.0255, -1.0544],
        [ 0.4708, -0.0932,  0.2063, -1.8165],
        [-0.0180,  0.6837, -1.8995,  0.4710],
        [ 0.2725, -0.2631, -0.7710, -0.0592]], device='cuda:0')]
tensor([[-2.0510, -1.7376,  0.6509,  0.1170],
        [-0.8056,  0.3006, -1.3536,  1.1864],
        [-1.1487,  0.4429, -0.4612,  0.5667],
        [ 1.5505,  0.4794,  0.2362, -0.3262],
        [ 1.4210,  0.9824,  1.0255, -1.054

True